In [1]:
import cv2
import numpy as np

In [2]:
def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    adaptive_thresh = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                           cv2.THRESH_BINARY, 11, 2)
    kernel = np.ones((3, 13), np.uint8)
    opened_mask = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_OPEN, kernel)
    contours, _ = cv2.findContours(opened_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    final_mask = np.zeros_like(opened_mask)
    for contour in contours:
        if cv2.contourArea(contour) >= 1500:
            cv2.drawContours(final_mask, [contour], -1, (255), thickness=cv2.FILLED)
    return final_mask

In [3]:
def angle_calculation(image):
    imageP = preprocess_image(image)
    edges = cv2.Canny(imageP, 50, 150)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)
    min_angle = 75
    max_angle = 105
    min_angle_rad = np.deg2rad(min_angle)
    max_angle_rad = np.deg2rad(max_angle)
    theta_sum = 0
    line_count = 0
    for line in lines:
        rho, theta = line[0]
        if min_angle_rad <= theta <= max_angle_rad:
            theta_sum += theta
            line_count += 1
    if  line_count > 0:
        theta_mean = theta_sum / line_count
        theta_mean_deg = np.rad2deg(theta_mean) 
        if theta_mean_deg > 90:
            return theta_mean_deg+5
        if theta_mean_deg < 90:
            return theta_mean_deg-2
        return theta_mean_deg 
    print("cane")
    return None

In [5]:
image_path = "provaGiorno.png"
image = cv2.imread(image_path)

print(angle_calculation(image))
img_matches = preprocess_image(image)

cv2.imshow('Matches', img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
